In [ ]:
!pip install ratsnlp

In [ ]:
!pip install kobert-transformers
!pip install transformers

In [ ]:
import torch
from ratsnlp.nlpbook.qa import QATrainArguments
from transformers import BertConfig, BertForQuestionAnswering
from kobert_transformers import get_tokenizer
from ratsnlp import nlpbook
from ratsnlp.nlpbook.qa import KorQuADV1Corpus, QADataset, QATask
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from sklearn.metrics import f1_score, accuracy_score
import os
import numpy as np

# 학습 (평가제외)

In [ ]:
import os
import torch
from ratsnlp.nlpbook.qa import QATrainArguments
from transformers import BertConfig, BertForQuestionAnswering
from kobert_transformers import get_tokenizer
from ratsnlp import nlpbook
from ratsnlp.nlpbook.qa import KorQuADV1Corpus, QADataset, QATask
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from sklearn.metrics import f1_score, accuracy_score
import numpy as np

# CUDA 메모리 설정
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

# Arguments 설정
args = QATrainArguments(
    pretrained_model_name="skt/kobert-base-v1",
    downstream_corpus_name="korquad-v1",
    downstream_model_dir="/home/km0228kr/KoBERT/model/model1285e64",
    max_seq_length=128,
    max_query_length=64,
    doc_stride=64,
    batch_size=16,  # 복수 GPU를 사용하므로 배치 크기를 늘림
    learning_rate=5e-5,  # learning rate 유지
    epochs=3,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
    downstream_corpus_root_dir="/home/km0228kr/Korpora"  # 다운로드 경로 수정
)

# Add this line to set the device
args.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# KoBERT 전용 토크나이저 사용
tokenizer = get_tokenizer()

# 모델 설정
pretrained_model_config = BertConfig.from_pretrained("skt/kobert-base-v1")
model = BertForQuestionAnswering.from_pretrained("skt/kobert-base-v1", config=pretrained_model_config)
model.to(args.device)

# 시드 설정 및 로깅
nlpbook.set_seed(args)
nlpbook.set_logger(args)

# 다운로드 경로 수정
os.makedirs(args.downstream_corpus_root_dir, exist_ok=True)

nlpbook.download_downstream_dataset(args)

# 데이터셋 준비
corpus = KorQuADV1Corpus()
train_dataset = QADataset(args=args, corpus=corpus, tokenizer=tokenizer, mode="train")
val_dataset = QADataset(args=args, corpus=corpus, tokenizer=tokenizer, mode="val")

# 데이터로더 설정
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=4,
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=4,
)

# 학습 태스크 설정
task = QATask(model, args)

# 트레이너 설정 (복수 GPU 사용)
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    dirpath=args.downstream_model_dir,
    save_top_k=1,
    monitor='val_loss',
    mode='min',
    filename='{epoch}-{val_loss:.2f}'
)

trainer = Trainer(
    max_epochs=args.epochs,
    strategy='dp',  # 분산 학습 전략을 dp로 설정
    gpus=torch.cuda.device_count(),  # 사용 가능한 모든 GPU 사용
    callbacks=[checkpoint_callback],
    progress_bar_refresh_rate=20
)

# 학습
trainer.fit(task, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

# 모델 저장
os.makedirs(args.downstream_model_dir, exist_ok=True)
model_path = os.path.join(args.downstream_model_dir, "pytorch_model.bin")
torch.save(model.state_dict(), model_path)
vocab_path = os.path.join(args.downstream_model_dir, "vocab.txt")
tokenizer.save_vocabulary(args.downstream_model_dir)
config_path = os.path.join(args.downstream_model_dir, "config.json")
pretrained_model_config.save_pretrained(args.downstream_model_dir)

print(f"Model saved to {model_path}")
print(f"Tokenizer vocabulary saved to {vocab_path}")
print(f"Config saved to {config_path}")

# 평가 함수
def evaluate_model(model, dataloader, tokenizer):
    model.eval()
    pred_answers = []
    true_answers = []

    for batch in dataloader:
        inputs = {
            "input_ids": batch["input_ids"].to(args.device),
            "attention_mask": batch["attention_mask"].to(args.device),
            "token_type_ids": batch["token_type_ids"].to(args.device),
        }
        with torch.no_grad():
            outputs = model(**inputs)

        start_logits = outputs.start_logits.detach().cpu().numpy()
        end_logits = outputs.end_logits.detach().cpu().numpy()

        for i in range(start_logits.shape[0]):
            start_idx = np.argmax(start_logits[i])
            end_idx = np.argmax(end_logits[i])
            pred_answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(batch["input_ids"][i][start_idx:end_idx+1]))
            true_answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(batch["input_ids"][i][batch["start_positions"][i]:batch["end_positions"][i]+1]))
            pred_answers.append(pred_answer)
            true_answers.append(true_answer)

    return pred_answers, true_answers

# F1 Score, Accuracy, Exact Match 계산
def calculate_metrics(pred_answers, true_answers):
    f1 = f1_score(true_answers, pred_answers, average='macro')
    accuracy = accuracy_score(true_answers, pred_answers)
    exact_match = sum([1 if pred == true else 0 for pred, true in zip(pred_answers, true_answers)]) / len(true_answers)
    return f1, accuracy, exact_match

'''# 모델 평가
pred_answers, true_answers = evaluate_model(model, val_dataloader, tokenizer)
f1, accuracy, exact_match = calculate_metrics(pred_answers, true_answers)

print(f"F1 Score: {f1}")
print(f"Accuracy: {accuracy}")
print(f"Exact Match: {exact_match}")'''


# 평가

In [ ]:
'''
평가용
"/content/drive/MyDrive/공모전/kobert_323e"
32
3e-5
'''
import torch
from sklearn.metrics import f1_score, accuracy_score
from kobert_transformers import get_tokenizer
import numpy as np
from ratsnlp.nlpbook.qa import QATrainArguments
from transformers import BertForQuestionAnswering
from ratsnlp import nlpbook
from ratsnlp.nlpbook.qa import KorQuADV1Corpus, QADataset
from torch.utils.data import DataLoader, SequentialSampler

# 학습 파라미터 설정
args = QATrainArguments(
    pretrained_model_name="skt/kobert-base-v1",
    downstream_corpus_name="korquad-v1",
    downstream_model_dir="/home/km0228kr/KoBERT/model/model1285e64",
    max_seq_length=128,
    max_query_length=64,
    doc_stride=64,
    batch_size=16,  # 배치 크기 설정
    learning_rate=5e-5,  # 학습률 설정
    epochs=3,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
    downstream_corpus_root_dir="./Korpora"  # 변경된 경로
)

# 모델 불러오기
model_path = "/home/km0228kr/KoBERT/model/model1285e64/pytorch_model.bin"
model = BertForQuestionAnswering.from_pretrained("skt/kobert-base-v1")
model.load_state_dict(torch.load(model_path, map_location=torch.device("cuda" if torch.cuda.is_available() else "cpu")))

# 디바이스 설정 (GPU 사용 가능 시 GPU 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# KoBERT 전용 토크나이저 사용
tokenizer = get_tokenizer()

# 평가 함수 정의
def evaluate_model(model, dataloader, tokenizer):
    model.eval()
    model.to(device)
    pred_answers = []
    true_answers = []

    for batch in dataloader:
        inputs = {
            "input_ids": batch["input_ids"].to(device),
            "attention_mask": batch["attention_mask"].to(device),
            "token_type_ids": batch["token_type_ids"].to(device),
        }
        with torch.no_grad():
            outputs = model(**inputs)

        start_logits = outputs.start_logits.detach().cpu().numpy()
        end_logits = outputs.end_logits.detach().cpu().numpy()

        for i in range(start_logits.shape[0]):
            start_idx = np.argmax(start_logits[i])
            end_idx = np.argmax(end_logits[i])
            pred_answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(batch["input_ids"][i][start_idx:end_idx+1]))
            true_answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(batch["input_ids"][i][batch["start_positions"][i]:batch["end_positions"][i]+1]))
            pred_answers.append(pred_answer)
            true_answers.append(true_answer)

    return pred_answers, true_answers

# F1 Score, Accuracy, Exact Match 계산 함수 정의
def calculate_metrics(pred_answers, true_answers):
    f1 = f1_score(true_answers, pred_answers, average='macro')
    accuracy = accuracy_score(true_answers, pred_answers)
    exact_match = sum([1 if pred == true else 0 for pred, true in zip(pred_answers, true_answers)]) / len(true_answers)
    return f1, accuracy, exact_match

# 시드 설정 및 로깅
nlpbook.set_seed(args)
nlpbook.set_logger(args)

# 다운로드 경로 수정
os.makedirs(args.downstream_corpus_root_dir, exist_ok=True)

nlpbook.download_downstream_dataset(args)

# 데이터셋 준비
corpus = KorQuADV1Corpus()
train_dataset = QADataset(args=args, corpus=corpus, tokenizer=tokenizer, mode="train")
val_dataset = QADataset(args=args, corpus=corpus, tokenizer=tokenizer, mode="val")

# 데이터로더 설정 (val_dataloader를 올바르게 설정)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=4,
)

# 모델 평가
pred_answers, true_answers = evaluate_model(model, val_dataloader, tokenizer)
f1, accuracy, exact_match = calculate_metrics(pred_answers, true_answers)

print(f"F1 점수: {f1}")
print(f"정확도: {accuracy}")
print(f"정확 일치: {exact_match}")
import json
import torch
from sklearn.metrics import f1_score
from kobert_transformers import get_tokenizer
import numpy as np
from ratsnlp.nlpbook.qa import QATrainArguments
from transformers import BertForQuestionAnswering
from ratsnlp.nlpbook.qa import KorQuADV1Corpus, QADataset
from torch.utils.data import DataLoader, SequentialSampler
import os

# 평가에 필요한 파라미터 설정
args = QATrainArguments(
    pretrained_model_name="skt/kobert-base-v1",
    downstream_corpus_name="korquad-v1",
    downstream_model_dir="/home/km0228kr/KoBERT/model/model1282e",
    max_seq_length=128,
    max_query_length=64,
    doc_stride=128,
    batch_size=16,
    learning_rate=2e-5,
    epochs=3,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
    downstream_corpus_root_dir="./Korpora"
)

# 모델 불러오기
model_path = "/home/km0228kr/KoBERT/model/model1282e/pytorch_model.bin"
model = BertForQuestionAnswering.from_pretrained("skt/kobert-base-v1")
model.load_state_dict(torch.load(model_path, map_location=torch.device("cuda" if torch.cuda.is_available() else "cpu")))

# 디바이스 설정 (GPU 사용 가능 시 GPU 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# KoBERT 전용 토크나이저 사용
tokenizer = get_tokenizer()

# 데이터셋 준비
corpus = KorQuADV1Corpus()
train_dataset = QADataset(args=args, corpus=corpus, tokenizer=tokenizer, mode="train")
val_dataset = QADataset(args=args, corpus=corpus, tokenizer=tokenizer, mode="val")

# 데이터로더 설정 (val_dataloader를 올바르게 설정)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=4,
)

# 평가 함수
def evaluate(model, dataloader, device):
    model.eval()
    all_predictions = []
    all_true_labels = []
    
    for batch in dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        token_type_ids = batch["token_type_ids"].to(device)
        start_positions = batch["start_positions"].to(device)
        end_positions = batch["end_positions"].to(device)

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
            start_logits = outputs.start_logits
            end_logits = outputs.end_logits

        start_preds = torch.argmax(start_logits, dim=-1)
        end_preds = torch.argmax(end_logits, dim=-1)

        for i in range(len(start_positions)):
            all_predictions.append((start_preds[i].item(), end_preds[i].item()))
            all_true_labels.append((start_positions[i].item(), end_positions[i].item()))

    # EM 및 F1 스코어 계산
    exact_matches = 0
    f1_total = 0

    for pred, true in zip(all_predictions, all_true_labels):
        pred_start, pred_end = pred
        true_start, true_end = true

        if pred_start == true_start and pred_end == true_end:
            exact_matches += 1

        pred_tokens = set(range(pred_start, pred_end + 1))
        true_tokens = set(range(true_start, true_end + 1))

        common_tokens = pred_tokens.intersection(true_tokens)

        if len(common_tokens) == 0:
            f1_total += 0
        else:
            precision = len(common_tokens) / len(pred_tokens)
            recall = len(common_tokens) / len(true_tokens)
            f1_total += 2 * (precision * recall) / (precision + recall)

    exact_match_score = exact_matches / len(all_true_labels)
    f1_score = f1_total / len(all_true_labels)

    print(f"Exact Match (EM): {exact_match_score}")
    print(f"F1 Score: {f1_score}")

# 평가 실행
evaluate(model, val_dataloader, device)